Notebook to predict N+1 command where N is 1..4 and command sets can be up to len 5

Construct Graph Function

In [95]:
from graph import Node

def construct_graph(command_list, command_dict={}):

    filter_empty = lambda x: (len(x) > 0)
    cur_node = None
    child_node = None
    
    for session in command_list:
        # remove commands of length 0 ( not good to modify list within loop )
        
        session = list(filter(filter_empty, session))

        try: 
            first_cmd = session[0]
        except Exception as inst:
            print(session)
            continue

        program = session[0].split()[0]

        if command_dict.get(program) is None:
            cur_node = Node(program=program, frequency=1)
            command_dict[program] = cur_node 
        else:
            cur_node = command_dict.get(program)
            cur_node.frequency += 1

        if cur_node.commands.get(first_cmd) is None:
            cur_node.commands[first_cmd] = 1
        else:
            cur_node.commands[first_cmd] += 1

        for cmd in range(1, len(session)):

            if session[cmd].isprintable() is False:
                break
            else:
                program = session[cmd].split()[0]

            if cur_node.children.get(program) is None:
                child_node = Node(program = program, frequency=1)
                cur_node.children[program] = child_node
            else:
                child_node = cur_node.children.get(program)
                child_node.frequency += 1
            
            if child_node.commands.get(session[cmd]) is None:
                child_node.commands[session[cmd]] = 1
            else:
                child_node.commands[session[cmd]] += 1
            
            cur_node = child_node
        
                
        
    return command_dict


Get Prediction Function

In [97]:
def get_prediction(command_list: list[str], graph, result_size=5, graph_depth=4):
    if len(command_list) == 0:
        return None

    commands = command_list[:graph_depth]
    previous_command = command_list[graph_depth]

    if len(commands) == 0:
        return None

    program = commands[0].split()[0]
    
    if graph.get(program) is None:
        return None
    else:
        node = graph[program]

    for command in commands[1:]:
                
        program = command.split()[0]
        if node.children.get(program) is not None:
            node = node.children[program]
        else:
            return None

    return node.get_prediction(previous_command, num_to_return=result_size)

Accuracy Function

In [110]:
from thefuzz import fuzz
from thefuzz import process
from sklearn.model_selection import train_test_split

import pprint
pp = pprint.PrettyPrinter(indent=4)

def append_list(lst, results, commands, graph_depth):
    lst.append({"Results": results, "Expected": commands[graph_depth], "Command Sequence": commands})

def get_accuracy(command_subsets, fail_list = [], succeed_list = [], correct_15_not_5 = [], print_fails=False, graph_depth=4):
    train_data, test_data = train_test_split(command_subsets, test_size=0.2, train_size=0.8, random_state=42)

    train_graph = construct_graph(train_data)

    # filter out nodes that can't get prediction because not enough length so don't contribute and will cause error
    test_data = [x for x in test_data if len(x) > graph_depth]
    test_size = len(test_data)

    return_5 = 5
    return_15 = 15

    correct = 0
    correct_with_15 = 0
    first_prediction = 0
    has_prediction = 0
    incorrect = 0
    none_count = 0

    for commands in test_data:
        results = get_prediction(commands, train_graph, return_5, graph_depth)
        results_15 = get_prediction(commands, train_graph, return_15, graph_depth)
        
        prev_correct = correct

        if results is not None:
            has_prediction += 1

            for i in range(len(results)):
                if fuzz.ratio(results[i][0], commands[graph_depth]) > 85:
                    correct += 1

                    if i == 0:
                        first_prediction += 1

                    if correct <= 100:
                        append_list(succeed_list, results, commands, graph_depth)

                    break

            if prev_correct == correct:
                incorrect += 1

            for i in range(len(results_15)):
                if fuzz.ratio(results_15[i][0], commands[graph_depth]) > 85:
                    correct_with_15 += 1

                    if correct != correct_with_15 and correct_with_15 <= 100:
                        append_list(correct_15_not_5, results_15, commands, graph_depth)

                    break

        else:
            none_count += 1

        if prev_correct == correct and print_fails:
            append_list(fail_list, results, commands, graph_depth)  
                      
    return 'Correct Proportion: {:.2f}% |\n Correct in 15 not 5: {:.2f}% |\n Has Prediction and is Correct: {:.2f}% |\n Incorrect Proportion: {:.2f}% |\n None Proportion: {:.2f}% |\n First Prediction: {:.2f}%'.format(100 * correct/test_size, 100 * correct_with_15/test_size, 100 * correct/has_prediction, 100 * incorrect/test_size, 100 * none_count/test_size, 100 * first_prediction/test_size)

In [99]:
from parse import Parser

parser = Parser()

100%|██████████| 56/56 [00:07<00:00,  7.03it/s]


All Commands Parse and Replace Args

In [100]:
science_session = parser.parse_commands_per_session(parser.scientists_files)
science_session = parser.parse_commands_into_subsets_sliding_window(science_session, 5)
science_session = parser.replace_args_nested(science_session)

experienced_session = parser.parse_commands_per_session(parser.experienced_files)
experienced_session = parser.parse_commands_into_subsets_sliding_window(experienced_session, 5)
experienced_session = parser.replace_args_nested(experienced_session)

non_session = parser.parse_commands_per_session(parser.non_programmers_files)
non_session = parser.parse_commands_into_subsets_sliding_window(non_session, 5)
non_session = parser.replace_args_nested(non_session)

novice_session = parser.parse_commands_per_session(parser.novice_files)
novice_session = parser.parse_commands_into_subsets_sliding_window(novice_session, 5)
novice_session = parser.replace_args_nested(novice_session)

100%|██████████| 56/56 [00:08<00:00,  6.49it/s]


In [101]:
from random import sample
import math

science_session_sample = parser.parse_commands_per_session(sample(parser.scientists_files, math.floor(0.2 * len(parser.scientists_files))))
science_session_sample = parser.parse_commands_into_subsets_sliding_window(science_session_sample, 5)
science_session_sample = parser.replace_args_nested(science_session_sample)

100%|██████████| 10/10 [00:03<00:00,  2.59it/s]


In [109]:
fail_list = []
succeed_list = []
print("Scientist session:\n", get_accuracy(science_session_sample, fail_list=fail_list, print_fails=True, succeed_list=succeed_list, graph_depth=1))

18496
18496
Scientist session:
 Correct Proportion: 74.12% |
 Correct in 15 not 5: 84.91% |
 Has Prediction and is Correct: 74.16% |
 Incorrect Proportion: 25.82% |
 None Proportion: 0.06% |
 First Prediction: 47.52%


In [89]:
fail_list

[{'Results': None,
  'Expected': 'pdp60 $0',
  'Command Sequence': ['pdplk $0 $1 $2 $3.01.1 $3.01.2',
   'pdplk $0 $1 $2 $3.05 $3.01.1 $3.01.2',
   'nm11 $0.pout > $1.t',
   'e $1.t',
   'pdp60 $0']},
 {'Results': [('pdpdb', 1925)],
  'Expected': 'pdpdb $0',
  'Command Sequence': ['dump $0 $1', 'e $0.t', 'dump $0 $1', 'e', 'pdpdb $0']},
 {'Results': [('l', 25)],
  'Expected': 'fred $0',
  'Command Sequence': ['lpq -Pip',
   'lprm -Pip $0',
   'lpq -Pip',
   'fred $0',
   'fred $0']}]

Accuracy predicting N+1 command from N commands with all commands in graph

In [111]:
for i in range(1, 5):
    print("Accuracy predicting command {} from commands {} through {}:\n".format(i+1, 1, i))

    print("Scientist session:\n", get_accuracy(science_session, graph_depth=i))
    print("Experienced session:\n", get_accuracy(experienced_session, graph_depth=i))
    print("Non programmer session:\n", get_accuracy(non_session, graph_depth=i))
    print("Novice session:\n", get_accuracy(novice_session, graph_depth=i))

  0%|          | 0/4 [00:00<?, ?it/s]

Accuracy predicting command 2 from commands 1 through 1:

Scientist session:
 Correct Proportion: 67.02% |
 Correct in 15 not 5: 78.54% |
 Has Prediction and is Correct: 67.06% |
 Incorrect Proportion: 32.91% |
 None Proportion: 0.07% |
 First Prediction: 43.22%
Experienced session:
 Correct Proportion: 66.73% |
 Correct in 15 not 5: 79.79% |
 Has Prediction and is Correct: 66.76% |
 Incorrect Proportion: 33.22% |
 None Proportion: 0.05% |
 First Prediction: 42.54%
Non programmer session:
 Correct Proportion: 67.08% |
 Correct in 15 not 5: 79.30% |
 Has Prediction and is Correct: 67.10% |
 Incorrect Proportion: 32.89% |
 None Proportion: 0.03% |
 First Prediction: 42.79%


 25%|██▌       | 1/4 [11:30<34:30, 690.22s/it]

Novice session:
 Correct Proportion: 78.51% |
 Correct in 15 not 5: 88.32% |
 Has Prediction and is Correct: 78.56% |
 Incorrect Proportion: 21.42% |
 None Proportion: 0.07% |
 First Prediction: 51.41%
Accuracy predicting command 3 from commands 1 through 2:

Scientist session:
 Correct Proportion: 79.83% |
 Correct in 15 not 5: 89.26% |
 Has Prediction and is Correct: 80.09% |
 Incorrect Proportion: 19.85% |
 None Proportion: 0.33% |
 First Prediction: 57.01%
Experienced session:
 Correct Proportion: 81.06% |
 Correct in 15 not 5: 89.87% |
 Has Prediction and is Correct: 81.30% |
 Incorrect Proportion: 18.64% |
 None Proportion: 0.30% |
 First Prediction: 59.23%
Non programmer session:
 Correct Proportion: 78.31% |
 Correct in 15 not 5: 86.87% |
 Has Prediction and is Correct: 78.49% |
 Incorrect Proportion: 21.46% |
 None Proportion: 0.23% |
 First Prediction: 55.92%


 50%|█████     | 2/4 [22:34<22:30, 675.24s/it]

Novice session:
 Correct Proportion: 86.63% |
 Correct in 15 not 5: 93.73% |
 Has Prediction and is Correct: 86.88% |
 Incorrect Proportion: 13.08% |
 None Proportion: 0.29% |
 First Prediction: 63.04%
Accuracy predicting command 4 from commands 1 through 3:

Scientist session:
 Correct Proportion: 82.86% |
 Correct in 15 not 5: 87.88% |
 Has Prediction and is Correct: 83.54% |
 Incorrect Proportion: 16.33% |
 None Proportion: 0.81% |
 First Prediction: 66.53%
Experienced session:
 Correct Proportion: 81.62% |
 Correct in 15 not 5: 85.64% |
 Has Prediction and is Correct: 82.48% |
 Incorrect Proportion: 17.33% |
 None Proportion: 1.05% |
 First Prediction: 65.88%
Non programmer session:
 Correct Proportion: 79.54% |
 Correct in 15 not 5: 84.85% |
 Has Prediction and is Correct: 80.02% |
 Incorrect Proportion: 19.86% |
 None Proportion: 0.60% |
 First Prediction: 60.93%


 75%|███████▌  | 3/4 [23:03<06:20, 380.01s/it]

Novice session:
 Correct Proportion: 86.96% |
 Correct in 15 not 5: 91.41% |
 Has Prediction and is Correct: 87.39% |
 Incorrect Proportion: 12.54% |
 None Proportion: 0.50% |
 First Prediction: 66.57%
Accuracy predicting command 5 from commands 1 through 4:

Scientist session:
 Correct Proportion: 42.04% |
 Correct in 15 not 5: 43.14% |
 Has Prediction and is Correct: 45.22% |
 Incorrect Proportion: 50.94% |
 None Proportion: 7.02% |
 First Prediction: 36.59%
Experienced session:
 Correct Proportion: 26.56% |
 Correct in 15 not 5: 27.26% |
 Has Prediction and is Correct: 29.35% |
 Incorrect Proportion: 63.93% |
 None Proportion: 9.52% |
 First Prediction: 22.19%
Non programmer session:
 Correct Proportion: 36.81% |
 Correct in 15 not 5: 37.81% |
 Has Prediction and is Correct: 39.62% |
 Incorrect Proportion: 56.09% |
 None Proportion: 7.11% |
 First Prediction: 29.81%


100%|██████████| 4/4 [23:10<00:00, 347.56s/it]

Novice session:
 Correct Proportion: 56.69% |
 Correct in 15 not 5: 58.19% |
 Has Prediction and is Correct: 59.52% |
 Incorrect Proportion: 38.55% |
 None Proportion: 4.76% |
 First Prediction: 45.37%


In [115]:
i = 4
fail_list = []

print("Scientist session:\n", get_accuracy(science_session, fail_list=fail_list, print_fails=True, succeed_list=succeed_list, graph_depth=i))
print("Experienced session:\n", get_accuracy(experienced_session, graph_depth=i))
print("Non programmer session:\n", get_accuracy(non_session, graph_depth=i))
print("Novice session:\n", get_accuracy(novice_session, graph_depth=i))

Scientist session:
 Correct Proportion: 42.05% |
 Correct in 15 not 5: 43.18% |
 Has Prediction and is Correct: 45.22% |
 Incorrect Proportion: 50.93% |
 None Proportion: 7.02% |
 First Prediction: 36.63%
Experienced session:
 Correct Proportion: 36.93% |
 Correct in 15 not 5: 37.80% |
 Has Prediction and is Correct: 40.10% |
 Incorrect Proportion: 55.15% |
 None Proportion: 7.93% |
 First Prediction: 31.22%
Non programmer session:
 Correct Proportion: 42.69% |
 Correct in 15 not 5: 43.91% |
 Has Prediction and is Correct: 45.49% |
 Incorrect Proportion: 51.16% |
 None Proportion: 6.16% |
 First Prediction: 34.66%
Novice session:
 Correct Proportion: 60.60% |
 Correct in 15 not 5: 62.16% |
 Has Prediction and is Correct: 63.31% |
 Incorrect Proportion: 35.12% |
 None Proportion: 4.28% |
 First Prediction: 48.26%


In [116]:
fail_list

[{'Results': [],
  'Expected': 'man $0',
  'Command Sequence': ['ls', 'rm $0', 'ls', "'rm' $0", 'man $0']},
 {'Results': [('cd $0../TW2', 496),
   ('cd $0', 480),
   ('ls', 288),
   ('emacs $0.p', 288),
   ('su $0', 280)],
  'Expected': 'lpr -Pplsinc $0',
  'Command Sequence': ['ls', 'emacs $0.c', 'cd', 'cd $0', 'lpr -Pplsinc $0']},
 {'Results': [],
  'Expected': 'man $0',
  'Command Sequence': ['mail',
   'mail',
   'hello',
   'readnews -x -n $0 -n $1.graphics -n $2.general',
   'man $0']},
 {'Results': None,
  'Expected': 'lines $0.l',
  'Command Sequence': ['e $0.r',
   'lines $0.l',
   'lines $0.l',
   'lines $0.l',
   'lines $0.l']},
 {'Results': [],
  'Expected': 'printenv',
  'Command Sequence': ['mc', '~/bin/mc', 'e $0', 'mc', 'printenv']},
 {'Results': [('vx', 7)],
  'Expected': 'ls',
  'Command Sequence': ['fg', 'pr -v $0.sl', 'pr $0.sl', 'kermit', 'ls']},
 {'Results': [],
  'Expected': 'pwd',
  'Command Sequence': ['ean', 'mail', 'page $0', 'ls -l $0', 'pwd']},
 {'Results':

Sampled Commands Parse and Replace Args

In [59]:
from random import sample
import math

subset_size = 5

science_session_sample = parser.parse_commands_per_session(sample(parser.scientists_files, math.floor(0.2 * len(parser.scientists_files))))
science_session_sample = parser.parse_commands_into_subsets_sliding_window(science_session_sample, 5)
science_session_sample = parser.replace_args_nested(science_session_sample)

experienced_session_sample = parser.parse_commands_per_session(sample(parser.experienced_files, math.floor(0.2 * len(parser.experienced_files))))
experienced_session_sample = parser.parse_commands_into_subsets_sliding_window(experienced_session_sample, 5)
experienced_session_sample = parser.replace_args_nested(experienced_session_sample)

non_session_sample = parser.parse_commands_per_session(sample(parser.non_programmers_files, math.floor(0.2 * len(parser.non_programmers_files))))
non_session_sample = parser.parse_commands_into_subsets_sliding_window(non_session_sample, 5)
non_session_sample = parser.replace_args_nested(non_session_sample)

novice_session_sample = parser.parse_commands_per_session(sample(parser.novice_files, math.floor(0.2 * len(parser.novice_files))))
novice_session_sample = parser.parse_commands_into_subsets_sliding_window(novice_session_sample, 5)
novice_session_sample = parser.replace_args_nested(novice_session_sample)

100%|██████████| 11/11 [00:01<00:00,  8.96it/s]


Accuracy predicting N+1 command from N commands with subset of data

In [112]:
from tqdm import tqdm

for i in tqdm(range(1, 5)):
    print("Accuracy Predicting Command {} from commands {} through {}:\n".format(i+1, 1, i))

    print("Scientist session sample:\n", get_accuracy(science_session_sample, graph_depth=i))
    print("Experienced session sample:\n", get_accuracy(experienced_session_sample, graph_depth=i))
    print("Non programmer session sample:\n", get_accuracy(non_session_sample, graph_depth=i))
    print("Novice session sample:\n", get_accuracy(novice_session_sample, graph_depth=i))

  0%|          | 0/4 [00:00<?, ?it/s]

Accuracy Predicting Command 2 from commands 1 through 1:

Scientist session sample:
 Correct Proportion: 68.81% |
 Correct in 15 not 5: 82.54% |
 Has Prediction and is Correct: 68.81% |
 Incorrect Proportion: 31.19% |
 None Proportion: 0.00% |
 First Prediction: 44.93%
Experienced session sample:
 Correct Proportion: 70.46% |
 Correct in 15 not 5: 82.42% |
 Has Prediction and is Correct: 70.46% |
 Incorrect Proportion: 29.54% |
 None Proportion: 0.00% |
 First Prediction: 44.51%
Non programmer session sample:
 Correct Proportion: 72.88% |
 Correct in 15 not 5: 84.91% |
 Has Prediction and is Correct: 72.88% |
 Incorrect Proportion: 27.12% |
 None Proportion: 0.00% |
 First Prediction: 46.52%


 25%|██▌       | 1/4 [14:46<44:18, 886.24s/it]

Novice session sample:
 Correct Proportion: 82.73% |
 Correct in 15 not 5: 92.47% |
 Has Prediction and is Correct: 82.73% |
 Incorrect Proportion: 17.27% |
 None Proportion: 0.00% |
 First Prediction: 51.90%
Accuracy Predicting Command 3 from commands 1 through 2:

Scientist session sample:
 Correct Proportion: 84.20% |
 Correct in 15 not 5: 93.33% |
 Has Prediction and is Correct: 84.21% |
 Incorrect Proportion: 15.79% |
 None Proportion: 0.01% |
 First Prediction: 60.52%
Experienced session sample:
 Correct Proportion: 83.73% |
 Correct in 15 not 5: 92.60% |
 Has Prediction and is Correct: 83.74% |
 Incorrect Proportion: 16.26% |
 None Proportion: 0.01% |
 First Prediction: 61.59%
Non programmer session sample:
 Correct Proportion: 86.02% |
 Correct in 15 not 5: 93.32% |
 Has Prediction and is Correct: 86.02% |
 Incorrect Proportion: 13.98% |
 None Proportion: 0.00% |
 First Prediction: 65.58%


 50%|█████     | 2/4 [15:18<12:47, 383.78s/it]

Novice session sample:
 Correct Proportion: 89.89% |
 Correct in 15 not 5: 96.25% |
 Has Prediction and is Correct: 89.89% |
 Incorrect Proportion: 10.11% |
 None Proportion: 0.00% |
 First Prediction: 66.82%
Accuracy Predicting Command 4 from commands 1 through 3:

Scientist session sample:
 Correct Proportion: 92.72% |
 Correct in 15 not 5: 97.28% |
 Has Prediction and is Correct: 92.72% |
 Incorrect Proportion: 7.28% |
 None Proportion: 0.01% |
 First Prediction: 74.85%
Experienced session sample:
 Correct Proportion: 92.54% |
 Correct in 15 not 5: 96.33% |
 Has Prediction and is Correct: 92.54% |
 Incorrect Proportion: 7.46% |
 None Proportion: 0.00% |
 First Prediction: 76.20%
Non programmer session sample:
 Correct Proportion: 93.66% |
 Correct in 15 not 5: 97.47% |
 Has Prediction and is Correct: 93.66% |
 Incorrect Proportion: 6.34% |
 None Proportion: 0.00% |
 First Prediction: 78.04%


 75%|███████▌  | 3/4 [15:23<03:31, 211.14s/it]

Novice session sample:
 Correct Proportion: 95.06% |
 Correct in 15 not 5: 98.90% |
 Has Prediction and is Correct: 95.06% |
 Incorrect Proportion: 4.94% |
 None Proportion: 0.00% |
 First Prediction: 73.70%
Accuracy Predicting Command 5 from commands 1 through 4:

Scientist session sample:
 Correct Proportion: 81.89% |
 Correct in 15 not 5: 84.08% |
 Has Prediction and is Correct: 82.17% |
 Incorrect Proportion: 17.77% |
 None Proportion: 0.34% |
 First Prediction: 69.53%
Experienced session sample:
 Correct Proportion: 81.36% |
 Correct in 15 not 5: 83.26% |
 Has Prediction and is Correct: 81.59% |
 Incorrect Proportion: 18.35% |
 None Proportion: 0.29% |
 First Prediction: 71.52%
Non programmer session sample:
 Correct Proportion: 85.71% |
 Correct in 15 not 5: 87.88% |
 Has Prediction and is Correct: 85.90% |
 Incorrect Proportion: 14.07% |
 None Proportion: 0.22% |
 First Prediction: 74.03%


100%|██████████| 4/4 [15:25<00:00, 231.35s/it]

Novice session sample:
 Correct Proportion: 90.95% |
 Correct in 15 not 5: 92.81% |
 Has Prediction and is Correct: 91.05% |
 Incorrect Proportion: 8.94% |
 None Proportion: 0.11% |
 First Prediction: 73.83%
